# Gender and Age Classification using CNNs


La classification automatique par âge et par sexe est devenue pertinente pour un nombre croissant d'applications, en particulier depuis l'essor des plateformes sociales et des médias sociaux. 

Dans ce projet Python, nous utiliserons le deep learning pour identifier avec précision le sexe et l'âge d'une personne à partir d'une seule image d'un visage. 

Le sexe prédit peut être l'un des Hommes et  Féminin, et l'âge prévu peut être l'une des plages suivantes: (0 - 2), (4 - 6), (8 - 12), (15 - 20) , (25 - 32), (38 - 43), (48 - 53), (60 - 100) (8 nœuds dans la couche softmax finale). Il est très difficile de deviner avec précision un âge exact à partir d'une seule image en raison de facteurs tels que le maquillage, l'éclairage, les obstructions et les expressions faciales. Et donc, nous en faisons un problème de classification au lieu d'en faire un problème de régression.

Pour aborder le projet python, nous allons:
1.   Détecter les visages
2.   Classer en homme / femme
3.   Classez-vous dans l'une des 8 tranches d'âge
4.   Mettez les résultats sur l'image et affichez-la





**DATASET**

Pour ce projet Python, nous utiliserons l'ensemble de données Adience.

Cet ensemble de données sert de référence pour les photos de visage et comprend diverses conditions d'imagerie du monde réel telles que le bruit, l'éclairage, la pose et l'apparence. 

Les images ont été collectées à partir d'albums Flickr et distribuées sous la licence Creative Commons (CC). Il a un total de 26 580 photos de 2 284 sujets dans huit tranches d'âge (comme mentionné ci-dessus) et mesure environ 1 Go. 

Les modèles que nous utiliserons ont été formés sur cet ensemble de données.

# Age Gender classification from a Youtube Video

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Mounted at /gdrive
/gdrive


In [ ]:
!pip install pafy
!pip install youtube_dl

pafy: la bibliothèque Pafy est utilisée pour récupérer le contenu et les métadonnées YouTube (tels que le titre, la note, le nombre de vues, la durée, la note, l'auteur, la vignette, les mots clés, etc.). 

In [ ]:
import pafy

url = 'https://www.youtube.com/watch?v=cWc7vYjgnTs'
vPafy = pafy.new(url)
print(vPafy.title)
print(vPafy.rating)
print(vPafy.viewcount)
print(vPafy.author)
print(vPafy.length)

Luciano Pavarotti sings "Nessun dorma" from Turandot (The Three Tenors in Concert 1994)
4.9185781
23187579
Warner Classics
195


1. Get the video URL from YouTube.
2. Face detection with Haar cascades
3. Gender Recognition with CNN
4. Age Recognition with CNN

1 .prototxt — The definition of CNN goes in here. This file defines the layers in the neural network, each layer’s inputs, outputs and functionality.

2 .caffemodel — This contains the information of the trained neural network (trained model).

Étape 1: Importez toutes les bibliothèques requises

In [ ]:
import cv2
import numpy as np

Étape 2: Obtenez l'URL de la vidéo Youtube et créez un objet «play» qui contient la meilleure résolution de la vidéo au format webm / mp4

In [ ]:
url = 'https://www.youtube.com/watch?v=c07IsbSNqfI&feature=youtu.be'
vPafy = pafy.new(url)
play = vPafy.getbest(preftype="mp4")

Étape 3: Souvent, nous devons capturer le flux en direct avec une caméra. OpenCV fournit une interface très simple à cela. Nous pouvons capturer la vidéo de la caméra, la convertir en vidéo en niveaux de gris et l'afficher. Juste une tâche simple pour commencer.

Pour capturer une vidéo, vous devez créer un objet de capture vidéo. Son argument peut être soit l'index du périphérique, soit le nom d'un fichier vidéo. L'index de l'appareil est juste le numéro pour spécifier quelle caméra. Normalement, une caméra sera connectée (comme dans mon cas). Donc je passe simplement 0 (ou -1). Vous pouvez sélectionner la deuxième caméra en passant 1 et ainsi de suite. Après cela, vous pouvez capturer image par image

In [ ]:
cap = cv2.VideoCapture(0) #if you are using webcam

Mais dans mon cas, je lis une URL de vidéo en ligne, pour cela, je transmettrai l'objet "play" à VideoCapture ()

In [ ]:
cap = cv2.VideoCapture(play.url)

Étape 4: à l'aide de set (), je définirai la hauteur et la largeur de notre image vidéo. cap.set (propId, value), ici 3 est le propertyId de width et 4 est pour Height

In [ ]:
cap.set(3, 480) #set width of the frame
cap.set(4, 640) #set height of the frame

False

Étape 5: Créez 3 listes distinctes pour stocker Model_Mean_Values, Age et Sexe.

In [ ]:
MODEL_MEAN_VALUES = (78.4263377603, 87.7689143744, 114.895847746)
age_list = ['(0, 2)', '(4, 6)', '(8, 12)', '(15, 20)', '(25, 32)', '(38, 43)', '(48, 53)', '(60, 100)']
gender_list = ['Male', 'Female']

Étape 6: J'ai défini une fonction pour charger le caffemodel et le prototxt du détecteur d'âge et de sexe, ce sont essentiellement des modèles CNN pré-entraînés qui feront la détection

In [ ]:
path_age_pro=r'/gdrive/My Drive/DL/age_deploy.prototxt'
path_age_coffemodel=r'/gdrive/My Drive/DL/age_net.caffemodel'
path_gender_pro=r'/gdrive/My Drive/DL/gender_deploy.prototxt'
path_gender_coffemodel=r'/gdrive/My Drive/DL/gender_net.caffemodel'
path_cascade=r'/gdrive/My Drive/DL/haarcascade_frontalface_alt.xml'

In [ ]:
def load_caffe_models():

 age_net = cv2.dnn.readNetFromCaffe(path_age_pro,path_age_coffemodel)
 gender_net = cv2.dnn.readNetFromCaffe(path_gender_pro, path_gender_coffemodel)

 return(age_net, gender_net)

Étape 7: Nous allons maintenant effectuer la détection des visages, la détection de l'âge et la détection du genre et pour cela, nous allons créer une fonction video_detector (age_net, gender_net) dans votre fonction principale et passer age_net et gender_net comme paramètres

In [ ]:
age_net, gender_net = load_caffe_models()



In [ ]:
def video_detector(age_net, gender_net):
  font = cv2.FONT_HERSHEY_SIMPLEX

while True:

  ret, image = cap.read()
       
  face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_alt.xml')
 
  gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  faces = face_cascade.detectMultiScale(gray, 1.1, 5)

if(len(faces)>0):
   print("Found {} faces".format(str(len(faces))))

for (x, y, w, h )in faces:
   cv2.rectangle(image, (x, y), (x+w, y+h), (255, 255, 0), 2)

#Get Face 
   face_img = image[y:y+h, h:h+w].copy()
   blob = cv2.dnn.blobFromImage(face_img, 1, (227, 227), MODEL_MEAN_VALUES, swapRB=False)

#Predict Gender
   gender_net.setInput(blob)
   gender_preds = gender_net.forward()
   gender = gender_list[gender_preds[0].argmax()]
   print("Gender : " + gender)

#Predict Age
   age_net.setInput(blob)
   age_preds = age_net.forward()
   age = age_list[age_preds[0].argmax()]
   print("Age Range: " + age)

overlay_text = "%s %s" % (gender, age)
cv2.putText(image, overlay_text, (x, y), font, 1, (255, 255, 255), 2, cv2.LINE_AA)

cv2.imshow('frame', image)  
#0xFF is a hexadecimal constant which is 11111111 in binary.
if cv2.waitKey(1) & 0xFF == ord('q'): 
   break

error: ignored

In [ ]:
video_detector(age_net, gender_net)

Étape 8: Lisez l'objet cap créé à partir de VideoCapture () à l'étape 3.

cap.read () renvoie un booléen (True / False). Si le cadre est lu correctement, il sera vrai.

Vous pouvez donc vérifier la fin de la vidéo en vérifiant cette valeur de retour.
Parfois, le capuchon peut ne pas avoir initialisé la capture. Dans ce cas, ce code affiche une erreur.
Vous pouvez vérifier s'il est initialisé ou non par la méthode cap.isOpened (). Si c'est vrai, OK. Sinon, ouvrez-le en utilisant cap.open ()

In [ ]:
ret, image = cap.read()

Étape 9: Convertissez l'image en image grise car le détecteur de visage OpenCV attend des images grises

In [ ]:
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

Étape 10: Chargez le modèle pre-built pour la détection faciale.

In [ ]:
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_alt.xml')

Étape 11: Maintenant, comment détecter un visage à partir d'une image à l'aide de CascadeClassifier?
Eh bien, encore une fois, CascadedClassifier d'OpenCV nous a simplifié la tâche grâce à detectMultiScale (), qui détecte exactement ce dont vous avez besoin

In [ ]:
detectMultiScale(image, scaleFactor, minNeighbors)

Voici les arguments qui devraient passer à detectMultiScale ().

C'est une fonction générale pour détecter des objets, dans ce cas, elle détectera les visages puisque nous avons appelé dans la cascade de visages. S'il trouve une face, il retourne une liste de positions de ladite face sous la forme «Rect (x, y, w, h).», Sinon, renvoie «Aucun».

Image: la première entrée est l'image en niveaux de gris.
scaleFactor: Cette fonction compense une fausse perception de taille qui se produit lorsqu'un visage semble être plus grand que l'autre simplement parce qu'il est plus proche de la caméra.
minNeighbours: algorithme de détection qui utilise une fenêtre mobile pour détecter les objets, il le fait en définissant combien d'objets se trouvent à proximité de l'actuel avant de pouvoir déclarer le visage trouvé.

In [ ]:
faces = face_cascade.detectMultiScale(gray, 1.1, 5)

Étape 12: Parcourez la liste des visages et dessinez des rectangles sur les visages humains dans la vidéo. Ici, nous recherchons essentiellement des visages, cassons les faces, leurs tailles et dessinons des rectangles

In [ ]:
for (x, y, w, h )in faces:
   cv2.rectangle(image, (x, y), (x+w, y+h), (255, 255, 0), 2)

# Get Face 
   face_img = image[y:y+h, h:h+w].copy()

Étape 13: OpenCV fournit une fonction pour faciliter le prétraitement des images pour la classification du deep learning: blobFromImage (). Il effectue:

Mean subtraction

Scaling

And optionally channel swapping

In [ ]:
blob = cv2.dnn.blobFromImage(image, scalefactor=1.0, size, mean, swapRB=True)


image: Il s'agit de l'image d'entrée que nous voulons prétraiter avant de la transmettre à notre réseau neuronal profond pour la classification.
facteur d'échelle: après avoir effectué une soustraction moyenne, nous pouvons éventuellement mettre à l'échelle nos images par un certain facteur. Cette valeur par défaut est 1.0 (c'est-à-dire pas de mise à l'échelle) mais nous pouvons également fournir une autre valeur. Il est également important de noter que le facteur d'échelle doit être de 1 / σ car nous multiplions en fait les canaux d'entrée (après soustraction de la moyenne) par le facteur d'échelle.
size: Ici, nous fournissons la taille spatiale attendue par le réseau de neurones convolutifs. Pour la plupart des réseaux de neurones de pointe actuels, il s'agit de 224 × 224, 227 × 227 ou 299 × 299.
signifie: ce sont nos valeurs moyennes de soustraction. Ils peuvent être un 3-tuple des moyens RVB ou ils peuvent être une valeur unique, auquel cas la valeur fournie est soustraite de chaque canal de l'image. Si vous effectuez une soustraction moyenne, assurez-vous de fournir les 3 tuples dans l'ordre (R, V, B), en particulier lorsque vous utilisez le comportement par défaut de swapRB = True.
swapRB: OpenCV suppose que les images sont dans l'ordre des canaux BGR; cependant, la valeur moyenne suppose que nous utilisons l'ordre RVB. Pour résoudre cet écart, nous pouvons permuter les canaux R et B de l'image en définissant cette valeur sur True. Par défaut, OpenCV effectue ce changement de chaîne pour nous.

In [ ]:
blob = cv2.dnn.blobFromImage(face_img, 1, (227, 227), MODEL_MEAN_VALUES, swapRB=False)

Etape 14: Predire le sexe

In [ ]:
#Predict Gender
gender_net.setInput(blob)
gender_preds = gender_net.forward()
gender = gender_list[gender_preds[0].argmax()]

Etape 15: Predire l'Age

In [ ]:
#Predict Age
age_net.setInput(blob)
age_preds = age_net.forward()
age = age_list[age_preds[0].argmax()]

Étape 16: Nous devons maintenant mettre du texte sur notre cadre de sortie en utilisant le module putText () d'OpenCV.
cv2.putText () prend les paramètres comme:

Données textuelles que vous souhaitez écrire
Positionnez les coordonnées de l'endroit où vous voulez les placer (c'est-à-dire le coin inférieur gauche où les données commencent).
Type de police (consultez la documentation cv2.putText () pour les polices prises en charge)
Échelle de police (spécifie la taille de la police)
des choses régulières comme la couleur, l'épaisseur, le type de ligne, etc. Pour une meilleure apparence, lineType = cv2.LINE_AA est recommandé.

In [ ]:
overlay_text = "%s %s" % (gender, age)
cv2.putText(image, overlay_text, (x, y), font, 1, (255, 255, 255), 2, cv2.LINE_AA)

Étape 17: Imprimez enfin votre sortie finale

In [ ]:
cv2.imshow('frame', image)

# Age Gender Classification From The Camera

**On repete les étapes en dessus mais cette fois on a la detection à partir d'une Camera**

In [ ]:
import cv2
import imutils
import time
import numpy as np

In [ ]:
cap = cv2.VideoCapture(0)

cap.set(3, 480) #set width
cap.set(4, 640) #set height

In [ ]:
MODEL_MEAN_VALUES = (78.4263377603, 87.7689143744, 114.895847746)
age_list = ['(0, 2)', '(4, 6)', '(8, 12)', '(15, 20)', '(25, 32)', '(38, 43)', '(48, 53)', '(60, 100)']
gender_list = ['Male', 'Female']

In [ ]:
def initialize_caffe_models():
	
	age_net = cv2.dnn.readNetFromCaffe(
		'data/deploy_age.prototxt', 
		'data/age_net.caffemodel')

	gender_net = cv2.dnn.readNetFromCaffe(
		'data/deploy_gender.prototxt', 
		'data/gender_net.caffemodel')

	return(age_net, gender_net)

In [ ]:
def read_from_camera(age_net, gender_net):
	font = cv2.FONT_HERSHEY_SIMPLEX

	while True:

		ret, image = cap.read()

		face_cascade = cv2.CascadeClassifier('data/haarcascade_frontalface_alt.xml')

		gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
		faces = face_cascade.detectMultiScale(gray, 1.1, 5)

		if(len(faces)>0):
			print("Found {} faces".format(str(len(faces))))

		for (x, y, w, h )in faces:
			cv2.rectangle(image, (x, y), (x+w, y+h), (255, 255, 0), 2)

			# Get Face 
			face_img = image[y:y+h, h:h+w].copy()
			blob = cv2.dnn.blobFromImage(face_img, 1, (227, 227), MODEL_MEAN_VALUES, swapRB=False)

			#Predict Gender
			gender_net.setInput(blob)
			gender_preds = gender_net.forward()
			gender = gender_list[gender_preds[0].argmax()]
			print("Gender : " + gender)

			#Predict Age
			age_net.setInput(blob)
			age_preds = age_net.forward()
			age = age_list[age_preds[0].argmax()]
			print("Age Range: " + age)

			overlay_text = "%s %s" % (gender, age)
			cv2.putText(image, overlay_text, (x, y), font, 1, (255, 255, 255), 2, cv2.LINE_AA)


		cv2.imshow('frame', image)

		if cv2.waitKey(1) & 0xFF == ord('q'):
			break

In [ ]:
if __name__ == "__main__":
	age_net, gender_net = initialize_caffe_models()

	read_from_camera(age_net, gender_net)

#  Predict Age Gender from pictures

**On repete les étapes precedent sur une image données**

In [ ]:
import cv2

In [ ]:
MODEL_MEAN_VALUES = (78.4263377603, 87.7689143744, 114.895847746)
age_list = ['(0, 2)', '(4, 6)', '(8, 12)', '(15, 20)', '(25, 32)', '(38, 43)', '(48, 53)', '(60, 100)']
gender_list = ['Male', 'Female']

In [ ]:
path_age_pro=r'/gdrive/My Drive/DL/age_deploy.prototxt'
path_age_coffemodel=r'/gdrive/My Drive/DL/age_net.caffemodel'
path_gender_pro=r'/gdrive/My Drive/DL/gender_deploy.prototxt'
path_gender_coffemodel=r'/gdrive/My Drive/DL/gender_net.caffemodel'
path_cascade=r'/gdrive/My Drive/DL/haarcascade_frontalface_alt.xml'

In [ ]:
def initialize_caffe_models():
	
	age_net = cv2.dnn.readNetFromCaffe(
		path_age_pro, 
		path_age_coffemodel)

	gender_net = cv2.dnn.readNetFromCaffe(
		path_gender_pro, 
		path_gender_coffemodel)

	return(age_net, gender_net)

In [ ]:
# read from image code
face=r'/gdrive/My Drive/DL/girl2.jpg'
def read_from_image(age_net,gender_net):
  font=cv2.FONT_HERSHEY_SIMPLEX
  image=cv2.imread(face)
  face_cascade=cv2.CascadeClassifier(path_cascade)
  gray=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
  faces=face_cascade.detectMultiscale(gray,1.1,5)
  if(len(faces)>0):
    print("found {} faces".format(str(len(faces))))
  for (x,y,w,h) in faces:
    cv2.rectangle(image,(x,y),(x+w,y+h),(255,255,0),2)
  #Get Face 
  face_img=image[y:y+h,h:h+w].copy()
  blob=cv2.dnn.blobFromImage(face_img,1,(227,227),MODEL_MEAN_VALUES,swapRB=False)
  #Predict Gender
  gender_net.setInput(blob)
  gender_preds=gender_net.forward()
  gender=gender_list[gender_preds[0].argmax]
  print("Gender:"+ gender)
  #Predict Age
  age_net.setInput(blob)
  age_preds=age_net.forward()
  age=age_list[age_preds[0].argmax]
  print("Age Range:"+ age)
  overlay_text="%S %S"%(gender,age)
  cv2.putText(image,overlay_text,(x,y),font,0.5,(100,100,255),2,cv2.LINE_AA)
  cv2.imshow("",image)
  cv2.waitKey(0)
  

In [ ]:
age_net, gender_net = initialize_caffe_models()


In [ ]:
read_from_image(age_net, gender_net)